# Question Augmentation (similarity)

A frequent source of difficulty in using Translator Knowledge Graphs is the traversal of similar nodes.  If two non-identical (but very similar) concepts exist, they may be independently associated with other information by individual KPs.  If a user is unaware of this subtlety, it is easy to miss highly relevant answers.

Here is a simple query: (gene)--(chemical)--(disease).  This query might be used to explain gene/disease relationships that are driven by some metabolic process.   For instance, there is a known association between the gene SLC34A1 and Fanconi Syndrome:

In [1]:
class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        print(json.dumps(r.json(),indent=2))
    
strider = Strider()

In [2]:
def get_view_url(returnanswer,robokop='robokop.renci.org'):
    """Given an answer in KGS v0.9 format, post the answer to robokop, and return a link that can be followed to
    view the answer in the UI"""
    view_post_url = f'https://{robokop}/api/simple/view/'
    view_post_response = requests.post(view_post_url, json=returnanswer)
    uid=view_post_response.json()
    view_url = f'https://{robokop}/simple/view/{uid}'
    return view_url

In [3]:
import requests
import json

## The relationship between the gene and disease is a known one.

In [4]:
question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:6569'},
                      {'id': 'n1', 'type':'disease', 'curie': 'MONDO:0001083'}],
             'edges': [ {'id': 'e0', 'source_id': 'n0', 'target_id': 'n1', 'type': 'contributes_to'}]}
message = {'message': {'query_graph': question}}

In [5]:
q = strider.send_message(message)
print(q)

3009de61-3b3d-4437-8515-287ef046c5e2


In [6]:
strider.query_result(q)

{
  "query_graph": {
    "nodes": [
      {
        "id": "n1",
        "curie": "MONDO:0001083",
        "type": "disease"
      },
      {
        "id": "n0",
        "curie": "NCBIGene:6569",
        "type": "gene"
      }
    ],
    "edges": [
      {
        "id": "e0",
        "type": "contributes_to",
        "source_id": "n0",
        "target_id": "n1"
      }
    ]
  },
  "knowledge_graph": null,
  "results": [
    {
      "node_bindings": [
        {
          "qg_id": "n0",
          "kg_id": "NCBIGene:6569"
        },
        {
          "qg_id": "n1",
          "kg_id": "MONDO:0001083"
        }
      ],
      "edge_bindings": [
        {
          "qg_id": "e0",
          "kg_id": "ffa8dda63bb560fb7ef670bec038e6eb"
        }
      ]
    }
  ]
}


## There is no chemical relating the gene to the disease

In [7]:
question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:6569'},
                       {'id':'n1', 'type':'chemical_substance'},
                      {'id': 'n2', 'type':'disease', 'curie':'MONDO:0001083'}],
             'edges': [ {'id': 'e0', 'source_id': 'n0', 'target_id': 'n1', 'type':'increases_degradation_of'},
                        {'id': 'e1', 'source_id': 'n1', 'target_id': 'n2', 'type':'related_to'}]}
message = {'message': {'query_graph': question}}

#strider = 'http://robokop.renci.org:5781/query'
#response = requests.post(strider,json=message)
#print(response.status_code)

In [8]:
q = strider.send_message(message)

In [10]:
strider.query_result(q)

{
  "query_graph": {
    "nodes": [
      {
        "id": "n1",
        "curie": null,
        "type": "chemical_substance"
      },
      {
        "id": "n2",
        "curie": "MONDO:0001083",
        "type": "disease"
      },
      {
        "id": "n0",
        "curie": "NCBIGene:6569",
        "type": "gene"
      }
    ],
    "edges": [
      {
        "id": "e0",
        "type": "increases_degradation_of",
        "source_id": "n0",
        "target_id": "n1"
      },
      {
        "id": "e1",
        "type": "related_to",
        "source_id": "n1",
        "target_id": "n2"
      }
    ]
  },
  "knowledge_graph": null,
  "results": []
}


## We can augment the question with similarity

In [12]:
qa_url = 'https://questionaugmentation.renci.org/node_expand'
rq = requests.post(qa_url,json=message)
new_questions = rq.json()
print(len(new_questions))
newq = new_questions[0]

#newq['query_graph']['edges'][-1]['type']='similar_to'
print(json.dumps(newq,indent=2))

1
{
  "query_graph": {
    "nodes": [
      {
        "id": "n0",
        "type": "gene",
        "curie": "NCBIGene:6569"
      },
      {
        "id": "n1",
        "type": "chemical_substance"
      },
      {
        "id": "n2",
        "type": "disease",
        "curie": "MONDO:0001083"
      },
      {
        "id": "sim_to_n1_0",
        "type": "chemical_substance"
      }
    ],
    "edges": [
      {
        "id": "e0",
        "source_id": "n0",
        "target_id": "n1",
        "type": "increases_degradation_of"
      },
      {
        "id": "e1",
        "source_id": "sim_to_n1_0",
        "target_id": "n2",
        "type": "related_to"
      },
      {
        "id": "sim_edge_0",
        "source_id": "n1",
        "target_id": "sim_to_n1_0"
      }
    ]
  }
}


In [46]:
newmessage = {'message': newq}
newmessage

{'message': {'query_graph': {'nodes': [{'id': 'n0',
     'type': 'gene',
     'curie': 'NCBIGene:6569'},
    {'id': 'n1', 'type': 'chemical_substance'},
    {'id': 'n2', 'type': 'disease', 'curie': 'MONDO:0001083'},
    {'id': 'sim_to_n1_0', 'type': 'chemical_substance'}],
   'edges': [{'id': 'e0',
     'source_id': 'n0',
     'target_id': 'n1',
     'type': 'increases_degradation_of'},
    {'id': 'e1',
     'source_id': 'sim_to_n1_0',
     'target_id': 'n2',
     'type': 'related_to'},
    {'id': 'sim_edge_0',
     'source_id': 'n1',
     'target_id': 'sim_to_n1_0',
     'type': 'similar_to'}]}}}

In [56]:
newmessage = {'message': {'query_graph': {'nodes': [{'id': 'n0',
     'type': 'gene',
     'curie': 'NCBIGene:6569'},
    {'id': 'n1', 'type': 'chemical_substance'},
    {'id': 'n2', 'type': 'disease', 'curie': 'MONDO:0001083'},
    {'id': 'sim_to_n1_0', 'type': 'chemical_substance'}],
   'edges': [{'id': 'e0',
     'source_id': 'n0',
     'target_id': 'n1',
     'type': 'increases_degradation_of'},
    {'id': 'e1',
     'source_id': 'sim_to_n1_0',
     'target_id': 'n2',
     'type': 'related_to'},
    {'id': 'sim_edge_0',
     'source_id': 'n1',
     'target_id': 'sim_to_n1_0',
     'type': 'derives_from'}]}}}

In [57]:
q = strider.send_message(newmessage)

In [37]:
strider.query_result(q)

{
  "detail": [
    {
      "loc": [
        "query",
        "query_id"
      ],
      "msg": "field required",
      "type": "value_error.missing"
    }
  ]
}
